In [10]:
import pandas as pd
import numpy as np

from tensorboardX import SummaryWriter

In [11]:
writer = SummaryWriter('runs/profiler')

In [12]:
df = pd.read_csv("/home/melinda/Examensarbete/DataSet/OriginalDataSet/Eclipse.csv", sep=';')

In [13]:
df = df[['bugID', 'sd', 'cl', 'pd', 'co', 'rp', 'os', 'bs', 'rs', 'pr', 'bsr']]